In [1]:
from syft.core.hooks import TorchHook
from syft.core.workers import VirtualWorker
import torch
import torch.nn as nn
from torch.autograd import Variable as Var
import torch.optim as optim
# this is our hook
hook = TorchHook()
local = hook.local_worker
remote = VirtualWorker(id=1,hook=hook)
local.add_worker(remote)

data = Var(torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = Var(torch.FloatTensor([[0],[0],[1],[1]]))

# model = nn.Linear(2,1)
model = Var(torch.zeros(2,1),requires_grad=True)



# generates grad objects
pred = data.mm(model)
loss = ((pred - target)**2).sum()
loss.backward()

Hooking into Torch...
Overloading complete.


In [2]:
# model.grad

In [3]:
# model.grad_backup = model.grad

In [4]:
# model.data

In [5]:
# model.grad.data

In [6]:
# FIRST
# model.data.id

In [7]:
# FIRST
# model.grad.data.id

In [13]:
model.owners

[<syft.core.workers.VirtualWorker id:1>]

In [14]:
model.data.owners

[<syft.core.workers.VirtualWorker id:1>]

In [15]:
model.grad.owners

[<syft.core.workers.VirtualWorker id:1>]

In [16]:
model.grad.data.owners

[<syft.core.workers.VirtualWorker id:1>]

In [17]:
model.is_pointer

True

In [18]:
model.data.is_pointer

True

In [19]:
model.grad.is_pointer

True

In [20]:
model.grad.data.is_pointer

True

In [12]:
model.send_(remote)

Variable containing:[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker id:1>]]

In [16]:
print(model.data)

[torch.FloatTensor with no dimension]



In [21]:
remote._objects

{80081310: [torch.FloatTensor - Locations:[1]],
 755672952: [torch.FloatTensor - Locations:[1]],
 3442370962: Variable containing:[torch.FloatTensor - Locations:[1]],
 4780697932: Variable containing:[torch.FloatTensor - Locations:[1]]}

In [22]:
model.id

3442370962

In [23]:
model.data.id

80081310

In [24]:
model.grad.id

4780697932

In [25]:
model.grad.data.id

755672952

In [6]:

opt = optim.SGD(params=model.parameters(),lr=0.1)

for i in range(10):
    
    # this does not work remotely
#     model.zero_grad()

    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    
    print(model.weight.data)
#     new_weight = model.weight.data - model.weight.grad
#     print(model.weight.grad)
    
    print(loss.get().data[0])

[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385
[torch.FloatTensor with no dimension]

1.4059107303619385


In [3]:
model.zero_grad()

In [4]:
model.weight.get()

Parameter containing:
-0.1936 -0.4076
[torch.FloatTensor of size 1x2]

In [5]:
model.weight.grad

Variable containing:
-33.6571 -17.9373
[torch.FloatTensor of size 1x2]